### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [306]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [307]:
import numpy as np
import matplotlib.pyplot as plt

In [308]:
cali = pd.read_csv('data/housing.csv')

In [309]:
cali.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [310]:
cali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [311]:
# separate data into target & independent variables 
x = cali.drop('median_house_value', axis=1) 
y = cali['median_house_value']
  
# split data into train and test set 
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0) 
print('Size of: ') 
print('Training Set x: ', x_train.shape) 
print('Training Set y: ', y_train.shape) 
print('Test Set x: ', x_test.shape) 
print('Test Set y: ', y_test.shape)

Size of: 
Training Set x:  (14447, 9)
Training Set y:  (14447,)
Test Set x:  (6193, 9)
Test Set y:  (6193,)


In [312]:
#Identify which features are categorical and which are numerical
categorical_features = ['ocean_proximity']

numerical_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

<pre>
1. Create a preprocessor with:
    a. SimpleImputer to fill in missing values (need to do this since 'fit' throws an error of NaN present after standardizing)
    b. Standardize the numerical data
    c. Perform PolynomialFeatures with degree of 3 (best MSE was obtained with degree=3)
    c. Perform One-Hot encoding on categorical data
2. Run LinearRegression 
using a pipeline
</pre>

In [301]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', make_pipeline(SimpleImputer(), StandardScaler(), PolynomialFeatures(degree=3, include_bias=False)), numerical_features),
        ('cat', make_pipeline(OneHotEncoder()), categorical_features)
    ])

# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipeline.fit(x_train, y_train)
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False))]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['ocean_proximity'])])),
                ('model', LinearRegression())])

In [303]:
# Evaluate the model with MSE for trained data and test data
from sklearn.metrics import mean_squared_error
y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f'MSE for training data: {mse_train}')
print(f'MSE for test data: {mse_test}')


MSE for training data: 3387868644.3552074
MSE for test data: 11726177953.136787


### Calculate the permutation importance using test dataset

In [304]:
# Calculate the permutation importance
perm_importance = permutation_importance(pipeline, x_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()[::-1] # [::-1] reverses the order of the array
# Print the feature importance scores
print("Feature importance scores (highest to lowest):")
for i in range(len(sorted_idx)):
    print(f"{x_test.columns[sorted_idx[i]]: >20}: {perm_importance.importances_mean[sorted_idx[i]]}")

Feature importance scores (highest to lowest):
          households: 263.7462578732616
      total_bedrooms: 261.4607373349603
         total_rooms: 10.63247980241174
          population: 7.601967725595726
            latitude: 2.12596356313931
           longitude: 1.3859674371850645
       median_income: 0.729396485418332
  housing_median_age: 0.059871326729991625
     ocean_proximity: 0.030874349347439177


In [305]:
# Using plotly express to visualize the feature importance
import plotly.express as px
fig = px.bar(x=x_test.columns[sorted_idx], y=perm_importance.importances_mean[sorted_idx])
fig.update_layout(title="Permutation Importance for each feature")
fig.update_xaxes(title_text="Feature")
fig.update_yaxes(title_text="Decrease in accuracy score")

fig.show()
